# Import libraries

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator, load_img
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pylab as plt
import os
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import f1_score,recall_score,accuracy_score,precision_score

In [10]:
from keras.layers import *
from keras.models import *
from keras.callbacks import *

In [13]:
!pip install protobuf
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/anaconda3/bin/python -m pip install --upgrade pip' command.


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  


In [11]:
!pip install keras-tuner

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/anaconda3/bin/python -m pip install --upgrade pip' command.


# Data
The data contains chest CT images.
The data contain 3 chest cancer types 
1. Adenocarcinoma
Adenocarcinoma of the lung: Lung adenocarcinoma is the most common form of lung cancer
Adenocarcinomas are found in several common cancers, including breast, prostate and colorectal.
Adenocarcinomas of the lung are found in the outer region of the lung
in glands that secrete mucus and help us breathe.
Symptoms include coughing, hoarseness, weight loss and weakness.

2. Large cell carcinoma
Large-cell undifferentiated carcinoma: Large-cell undifferentiated carcinoma lung cancer grows and spreads quickly and can
be found anywhere in the lung. This type of lung cancer usually accounts for 10
to 15 percent of all cases of NSCLC.
Large-cell undifferentiated carcinoma tends to grow and spread quickly.

3. Squamous cell carcinoma
Squamous cell: This type of lung cancer is found centrally in the lung,
where the larger bronchi join the trachea to the lung,
or in one of the main airway branches.
Squamous cell lung cancer is responsible for about 30 percent of all non-small
cell lung cancers, and is generally linked to smoking.

4. And the last folder is the normal CT-Scan images


# Path to Data folders

In [3]:
train_ds='./Chest CT-Scan images Dataset 3/Data/train'
test_ds='./Chest CT-Scan images Dataset 3/Data/test'
val_ds='./Chest CT-Scan images Dataset 3/Data/valid'

test_adenocarcinoma=os.listdir(test_ds+'/adenocarcinoma')
test_large=os.listdir(test_ds+'/large.cell.carcinoma')
test_squamous=os.listdir(test_ds+'/squamous.cell.carcinoma')

val_adenocarcinoma=os.listdir(val_ds+'/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib')
val_large=os.listdir(val_ds+'/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa')
val_squamous=os.listdir(val_ds+'/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa')

# Data size

In [4]:
print('trian size adenocarcinoma',len(os.listdir(train_ds+'/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib')))
print('trian size large',len(os.listdir(train_ds+'/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa')))
print('trian size squamous',len(os.listdir(train_ds+'/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa')))

trian size adenocarcinoma 195
trian size large 115
trian size squamous 155


In [5]:
print('test size adenocarcinoma ',len(test_adenocarcinoma))
print('test size large ',len(test_large))
print('test size squamous ',len(test_squamous))


test size adenocarcinoma  120
test size large  51
test size squamous  90


In [6]:
print('val size adenocarcinoma ',len(val_adenocarcinoma))
print('val size large ',len(val_large))
print('val size squamous ',len(val_squamous))

val size adenocarcinoma  23
val size large  21
val size squamous  15


# Augmentation:
ImageDataGenerator is a class that allows pre-processing of images,in order to format images before using them by training and drawing a model. This includes resizing, orientation and color corrections.

Explanation of Augmentation parameters:
Rescale 1./255 -is to transform every pixel value from range [0,255] -> [0,1]
width_shift_range -shift the image to the left or right(horizontal shifts)
height_shift_range -shift the image to up or down (vertically shifts).
shear_range is Shear Intensity (Shear angle in counter-clockwise direction in degrees)
rotation_range -randomly rotate images through any degree between 0 and 360 by providing an integer value in the rotation_range argument.

In [7]:
train_datagen= ImageDataGenerator(rescale = 1./255,validation_split = 0.2,
                                  
        rotation_range=5,
        width_shift_range=0.2,
        height_shift_range=0.2,
        #shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        vertical_flip=False,
        fill_mode='nearest')

valid_datagen= ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2)

test_datagen= ImageDataGenerator(rescale = 1./255)

# The use of Augmentation

In [8]:
train_dataset  = train_datagen.flow_from_directory(directory = train_ds,
                                                   target_size = (224,224),
                                                   class_mode = 'categorical',
                                                   batch_size = 64)

valid_dataset = valid_datagen.flow_from_directory(directory = test_ds,
                                                  target_size = (224,224),
                                                  class_mode = 'categorical',
                                                  batch_size = 64)

test_dataset = test_datagen.flow_from_directory(directory = val_ds,
                                                  target_size = (224,224),
                                                  class_mode = 'categorical',
                                                  batch_size = 64)

Found 465 images belonging to 3 classes.
Found 261 images belonging to 3 classes.
Found 59 images belonging to 3 classes.


# Print the Tag of datasets

In [9]:
import scipy
image_batch_train, label_batch_train = next(iter(train_dataset))
print("Image batch shape: ", image_batch_train.shape)
print("Label batch shape: ", label_batch_train.shape)
dataset_labels = sorted(train_dataset.class_indices.items(), key=lambda pair:pair[1])
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print('0: ',dataset_labels[0])
print('1: ',dataset_labels[1])
print('2: ',dataset_labels[2])


Image batch shape:  (64, 224, 224, 3)
Label batch shape:  (64, 3)
0:  Adenocarcinoma_Left.Lower.Lobe_T2_N0_M0_Ib
1:  Large.Cell.Carcinoma_Left.Hilum_T2_N2_M0_Iiia
2:  Squamous.Cell.Carcinoma_Left.Hilum_T1_N2_M0_Iiia


# HyperParameters Models

In [12]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(224,224,3)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.MaxPool2D(
        pool_size = hp.Choice('pool_1_size', values = [3,5])
      ),
    keras.layers.Dropout(0.3),
    keras.layers.Conv2D(
        filters=hp.Int('conv_3_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_3_kernel', values = [3,5]),
        activation='relu'
    ),

    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(3, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

# Using Random Search to find the optimal model

In [14]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=10,directory='output_new',project_name="Covid")

INFO:tensorflow:Reloading Oracle from existing project output_new/Covid/oracle.json
INFO:tensorflow:Reloading Tuner from output_new/Covid/tuner0.json


In [15]:
tuner_search.search(train_dataset ,epochs = 50, callbacks=[EarlyStopping(monitor='val_loss', mode='min',patience=10)], validation_data=(valid_dataset))

Trial 10 Complete [00h 31m 38s]
val_accuracy: 0.5670498013496399

Best val_accuracy So Far: 0.6321839094161987
Total elapsed time: 00h 31m 38s
INFO:tensorflow:Oracle triggered exit


In [16]:
tuner_search.results_summary()


Results summary
Results in output_new/Covid
Showing 10 best trials
Trial summary
Hyperparameters:
conv_1_filter: 32
conv_1_kernel: 5
conv_2_filter: 48
conv_2_kernel: 3
pool_1_size: 5
conv_3_filter: 32
conv_3_kernel: 3
dense_1_units: 64
learning_rate: 0.001
Score: 0.6321839094161987
Trial summary
Hyperparameters:
conv_1_filter: 80
conv_1_kernel: 3
conv_2_filter: 48
conv_2_kernel: 5
pool_1_size: 3
conv_3_filter: 32
conv_3_kernel: 3
dense_1_units: 48
learning_rate: 0.001
Score: 0.5823754668235779
Trial summary
Hyperparameters:
conv_1_filter: 64
conv_1_kernel: 5
conv_2_filter: 48
conv_2_kernel: 3
pool_1_size: 5
conv_3_filter: 64
conv_3_kernel: 3
dense_1_units: 112
learning_rate: 0.001
Score: 0.5670498013496399
Trial summary
Hyperparameters:
conv_1_filter: 96
conv_1_kernel: 3
conv_2_filter: 64
conv_2_kernel: 3
pool_1_size: 5
conv_3_filter: 64
conv_3_kernel: 3
dense_1_units: 64
learning_rate: 0.001
Score: 0.5478927493095398
Trial summary
Hyperparameters:
conv_1_filter: 48
conv_1_kernel: 3
co

# Taking the most optimal model

In [17]:
model=tuner_search.get_best_models(num_models=1)[0]

# save model

In [19]:
model.summary()
model.save('./Data 3/best_Model')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 220, 220, 32)      2432      
                                                                 
 conv2d_1 (Conv2D)           (None, 218, 218, 48)      13872     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 43, 43, 48)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 43, 43, 48)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 41, 41, 32)        13856     
                                                                 
 flatten (Flatten)           (None, 53792)             0         
                                                        

# calculation of predict images

In [20]:
y_pred=[]
model = keras.models.load_model('./Data 3/best_Model')
'''This function loads an image'''
def prepare_img(img, img_width, img_height,channels):
    img =  tf.keras.preprocessing.image.img_to_array(img)
    img = img.reshape(1, img_width, img_height, channels).astype('float32')
    img = img / 255.0
    return img

for img in test_adenocarcinoma:
    img = load_img('./Chest CT-Scan images Dataset/Data/test/adenocarcinoma/' + img ,target_size=(224, 224))
    img = prepare_img(img,224,224,3)
    predictions = model.predict(img)
    index_of_higher_value  = predictions[0].argmax()
    y_pred.append(predictions[0].argmax())

for img in test_large:
    img = load_img('./Chest CT-Scan images Dataset/Data/test/large.cell.carcinoma/' + img ,target_size=(224, 224))
    img = prepare_img(img,224,224,3)
    predictions = model.predict(img)
    index_of_higher_value  = predictions[0].argmax()
    y_pred.append(predictions[0].argmax())


for img in test_squamous:
    img = load_img('./Chest CT-Scan images Dataset/Data/test/squamous.cell.carcinoma/' + img ,target_size=(224, 224))
    img = prepare_img(img,224,224,3)
    predictions = model.predict(img)
    index_of_higher_value = predictions[0].argmax()
    y_pred.append(predictions[0].argmax())



# calculation of accuracy,recall,precision and  f1 score

In [21]:

y_test=[0]*len(test_adenocarcinoma)
y_test+=[1]*len(test_large)
y_test+=[2]*len(test_squamous)

print('Accuracy: ',round(accuracy_score(y_test, y_pred),2))
print('Recall: ',round(recall_score(y_test, y_pred,average='weighted'),2))
print('Precision: ',round(precision_score(y_test, y_pred,average='weighted'),2))
print('f1 score :',round(f1_score(y_test,y_pred, average='weighted'),2))

print(classification_report(y_test, y_pred))

Accuracy:  0.63
Recall:  0.63
Precision:  0.66
f1 score : 0.62
              precision    recall  f1-score   support

           0       0.65      0.63      0.64       120
           1       0.79      0.29      0.43        51
           2       0.59      0.82      0.69        90

    accuracy                           0.63       261
   macro avg       0.68      0.58      0.59       261
weighted avg       0.66      0.63      0.62       261



# Visualization of images

In [22]:
import ipyplot
images=[]
for i in os.listdir('./Data_jpg/a'):
  images.append('./Data_jpg/a/'+i)

labels=['adenocarcinoma','adenocarcinoma','adenocarcinoma','adenocarcinoma','adenocarcinoma','adenocarcinoma']
ipyplot.plot_class_tabs(images,labels, img_width=150)



images=[]
for i in os.listdir('./Data_jpg/l'):
  images.append('./Data_jpg/l/'+i)

labels=['large','large','large','large','large','large']
ipyplot.plot_class_tabs(images,labels, img_width=150)

images=[]
for i in os.listdir('./Data_jpg/s'):
  images.append('./Data_jpg/s/'+i)

labels=['squamous','squamous','squamous','squamous','squamous','squamous']
ipyplot.plot_class_tabs(images,labels, img_width=150)


images=[]
for i in os.listdir('./Data_jpg/n'):
  images.append('./Data_jpg/n/'+i)

labels=['normal','normal','normal','normal','normal','normal']
ipyplot.plot_class_tabs(images,labels, img_width=150)
